In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [7]:
# GPU 사용 가능한지 확인
device = torch.device("mps")

In [2]:
# Fast Gradient Sign Method (FGSM) 함수 정의
def fgsm_attack(image, epsilon, gradient):
    sign_gradient = gradient.sign()
    perturbed_image = image + epsilon * sign_gradient
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# 적대적 예제 생성을 위한 그래디언트 계산 함수 정의
def get_adversarial_examples(model, images, labels, epsilon):
    images.requires_grad_(True)
    outputs = model(images)
    loss = nn.CrossEntropyLoss()(outputs, labels)

    model.zero_grad()
    loss.backward()

    gradients = images.grad.data
    adversarial_examples = fgsm_attack(images, epsilon, gradients)
    return adversarial_examples

In [3]:
# 데이터 로드 및 전처리
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [8]:
# 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = nn.MaxPool2d(2)(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.LogSoftmax(dim=1)(x)
        return output

model = Net()
model.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [10]:
# 모델 컴파일
optimizer = optim.Adam(model.parameters())

# 하이퍼파라미터 설정
epsilon = 0.1
epochs = 5

# 적대적 훈련 시작
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # 데이터를 GPU로

Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [19]:
import torch.nn.functional as F

def adversarial_train(model, device, train_loader, optimizer, epoch, epsilon):
    model.train()  # 학습 모드로 전환
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # 적대적 예제 생성
        adversarial_data = get_adversarial_examples(model, data, target, epsilon)

        # 일반 데이터 및 적대적 예제를 사용하여 모델 학습
        optimizer.zero_grad()
        output = model(adversarial_data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

log_interval = 10  # 원하는 로그 출력 빈도 설정 (예: 10번의 배치마다 로그 출력)
# 적대적 훈련 실행
for epoch in range(1, epochs + 1):
    adversarial_train(model, device, train_loader, optimizer, epoch, epsilon)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.266969
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.040867
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.747249
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.289678
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.218882
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.066083
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.757916
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.700913
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.903277
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.861565
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.654392
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.784697
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.652854
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.631073
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.670632
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.744448
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.630793
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.626950
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.567920
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.424724
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.190235
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.082196
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.128352
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.157457
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.181500
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.215489
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.225254
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.206885
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.146389
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.206951
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.071837
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.168795
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.170115
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.151495
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.181919
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.081538
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.200782
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.205793
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.219970
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.192034


Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.309860
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.228248
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.173605
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.201490
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.130554
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.308701
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.176687
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.040097
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.052762
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.198934
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.036411
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.077626
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.172447
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.050191
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.131224
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.121874
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.083538
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.271065
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.150926
Train Epoch: 4 [44160/60000 (74%)]	Loss: 0.161781


In [22]:
# 적대적 예제 생성을 위한 그래디언트 계산 함수 정의 (기존과 동일)
def get_adversarial_examples(model, images, labels, epsilon, requires_grad=True):
    if requires_grad:
        images.requires_grad_(True)
    
    outputs = model(images)
    loss = nn.CrossEntropyLoss()(outputs, labels)

    if requires_grad:
        model.zero_grad()
        loss.backward()
        gradients = images.grad.data
        adversarial_examples = fgsm_attack(images, epsilon, gradients)
    else:
        with torch.no_grad():
            gradients = torch.zeros_like(images)
            adversarial_examples = fgsm_attack(images, epsilon, gradients)
            
    return adversarial_examples

# 평가를 위한 함수 정의 (적대적 예제를 생성하여 평가)
def test_adversarial(model, device, test_loader, epsilon):
    model.eval()  # 평가 모드로 전환
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # 데이터를 GPU로 옮김
            
            # 적대적 예제 생성 (requires_grad=False로 설정)
            adversarial_data = get_adversarial_examples(model, data, target, epsilon, requires_grad=False)
            
            # 적대적 예제에 대한 예측 수행
            output = model(adversarial_data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f"Adversarial Test Accuracy: {accuracy}%")

# 적대적 예제에 대한 테스트 실행
test_adversarial(model, device, test_loader, epsilon)


Adversarial Test Accuracy: 99.26%


In [21]:
def adversarial_train_v2(model, device, train_loader, optimizer, epoch, epsilon):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # 적대적 예제 생성
        adversarial_data = get_adversarial_examples(model, data, target, epsilon)
        
        # 원본 데이터와 적대적 예제를 함께 사용하여 모델 학습
        mixed_data = torch.cat((data, adversarial_data), 0)
        mixed_target = torch.cat((target, target), 0)
        
        optimizer.zero_grad()
        output = model(mixed_data)
        loss = F.cross_entropy(output, mixed_target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

# 적대적 훈련 실행
for epoch in range(1, epochs + 1):
    adversarial_train_v2(model, device, train_loader, optimizer, epoch, epsilon)
    test(model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.075503
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.033791
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.029738
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.151709
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.182816
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.019692
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.063801
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.199774
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.023967
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.127880
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.053368
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.042442
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.014378
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.047732
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.097890
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.017690
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.120921
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.056242
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.075228
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.047210
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.041225
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.085891
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.103754
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.017746
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.016566
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.013724
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.050932
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.028810
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.060847
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.024218
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.112816
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.034496
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.067145
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.127367
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.159063
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.137679
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.095394
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.066099
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.048459
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.015677


Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.029103
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.047017
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.052949
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.043274
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.021816
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.029685
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.066983
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.042674
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.019201
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.076667
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.074713
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.054587
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.068324
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.043276
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.085940
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.050653
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.039609
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.015833
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.029571
Train Epoch: 4 [44160/60000 (74%)]	Loss: 0.029556
